# Geolocation features handling
Once we already know how the stations behave each hour, on weekdays, we will add geolocation features to our model. Do they behave the same depending on how high in the city the stations are located? Is there any relationship between how close to the city centre they are? We will find this out in this notebook.

In [127]:
#Importing required libraries
import pandas as pd
import geopy.distance
import geocoder

In [75]:
#importing datetime_features file generated from 1_datetime
oct_features = pd.read_csv("..\\Dataset\\datetime_features.csv",encoding="utf_8",index_col='station_id')
#importing dataframe extracted from web scrapping Bicing website, thanks to Laurent Guerguy https://github.com/laurent-guerguy
coordinates_df = pd.read_csv("..\\Dataset\\bicing_ws_laurent.csv",encoding="utf_8")

In [76]:
#We only need 'station_id', latitude and longitude columns
coordinates_df = coordinates_df[['1','4','5']]

In [77]:
#We double check column '1' refers to station_id
coordinates_df['1'].unique()
print(f"The dataframe has the stations from its id number {coordinates_df['1'].min()} to the {coordinates_df['1'].max()}")

The dataframe has the stations from its id number 1 to the 496


In [78]:
coordinates_df = coordinates_df.rename(columns={"1":'station_id',"4":"lat","5":"long"})

In [79]:
coordinates_df = coordinates_df.groupby('station_id').mean()

In [114]:
oct_features_gps = pd.merge(oct_features, coordinates_df, on="station_id")

In [115]:
#Checking if we are missing something
oct_features_gps.isnull().sum()

00:00    0
01:00    0
02:00    0
03:00    0
04:00    0
05:00    0
06:00    0
07:00    0
08:00    0
09:00    0
10:00    0
11:00    0
12:00    0
13:00    0
14:00    0
15:00    0
16:00    0
17:00    0
18:00    0
19:00    0
20:00    0
21:00    0
22:00    0
23:00    0
lat      0
long     0
dtype: int64

### Calculating the distance of each station to the city centre
We will use Plaça Catalunya, Lat: 41.3870154, Long: 2.1700471 as the centre of the city and will caluclate the distance, in kilometres, of the station to the centre.

In [90]:
PlCat = (41.3870154, 2.170047)

In [123]:
#Iterating between stations to find the distance from Plaça Ctalunya
stations = range(oct_features_gps.shape[0])
distances = []
for s in stations:
    location = (oct_features_gps.iloc[s]['lat'],oct_features_gps.iloc[s]['long'])
    distances.append(geopy.distance.vincenty(PlCat, location).km)

C:\Users\jaume\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  """


In [125]:
#Adding distances as feature
oct_features_gps['DisttoCentre'] = distances

In [128]:
oct_features_gps.head()

,00:00,01:00,02:00,03:00,04:00,05:00,06:00,07:00,08:00,09:00,...,17:00,18:00,19:00,20:00,21:00,22:00,23:00,lat,long,DisttoCentre
station_id,,,,,,,,,,,,,,,,,,,,,
1,23.446767,23.829381,23.712932,23.849802,23.663094,20.892546,11.782609,4.006968,5.026528,6.000405,...,11.195500,11.897309,12.698009,15.794010,17.607353,20.164640,21.964028,41.397952,2.180042,1.474485
2,13.521435,14.101449,13.796012,13.482543,13.398019,11.883222,7.344203,3.507398,3.186860,4.340073,...,8.327506,6.906202,7.275058,8.782305,10.330977,12.419454,12.664574,41.395530,2.177060,1.112775
3,16.589744,16.688380,16.762821,16.660864,16.361255,16.761908,14.389657,7.888315,4.038994,3.496351,...,18.652326,19.573857,18.422139,16.544694,16.124531,16.378281,17.249603,41.394055,2.181299,1.223473
4,9.777668,9.742754,9.940635,10.385933,11.532634,14.071856,10.055665,4.903593,2.035700,2.817244,...,15.020523,15.481707,13.534940,11.529492,9.663069,9.610951,10.528793,41.393480,2.181555,1.200780
5,24.423558,24.391304,24.243590,23.869565,24.260236,23.059238,18.657115,9.327582,8.241005,10.146194,...,26.185644,30.289754,27.051282,27.036156,27.424775,26.778960,24.821970,41.391075,2.180223,0.963151


### Finding out the elevation of each station
Geocoder allows us finding the elevation of each coordinate

In [129]:
#Testing out
#g = geocoder.elevation([41.3870154, 2.170047])
#print(g.meters)

ValueError: Provide API Key